In [2]:
import pandas as pd


# Read the cleaned file into a DataFrame
df_cleaned = pd.read_csv('../data/bgg_rating_threshold.csv', sep=';', engine='c')

# Convert the 'bgg_user_rating' column to numeric
df_cleaned['bgg_user_rating'] = pd.to_numeric(df_cleaned['bgg_user_rating'], errors='coerce')

print("done")

done


In [3]:
import numpy as np
import scipy
# Create the matrix with games as rows and users as columns.
df_cleaned['bgg_user_rating'] = (df_cleaned['bgg_user_rating']*10).astype(np.uint8)
#user_game_matrix = df_cleaned.head(1000000).pivot(index='bgg_id', columns='bgg_user_name', values='bgg_user_rating')
user_game_matrix = df_cleaned.head(100000).set_index(['bgg_id', 'bgg_user_name'])['bgg_user_rating'].unstack('bgg_user_name',0)
sparse = scipy.sparse.csr_matrix(user_game_matrix.values)
# Check the shape and preview the matrix.
print(sparse.shape)

(11139, 1580)


In [9]:
from sklearn.decomposition import NMF, TruncatedSVD

model = TruncatedSVD(n_components=1580)


W = model.fit_transform(user_game_matrix)

In [7]:
# Convert the polars DataFrame to a NumPy array for numerical computations.
matrix_np = user_game_matrix.to_numpy()

# Reconstruct the approximation from the TruncatedSVD factors.
reconstructed = W @ model.components_

# Compute the Frobenius norm of the difference between the original matrix and the reconstruction.
error = np.linalg.norm(matrix_np - reconstructed, ord='fro')

# Compute the Frobenius norm of the original matrix.
norm_orig = np.linalg.norm(matrix_np, ord='fro')

# Calculate the relative reconstruction error.
relative_error = error / norm_orig

print("Relative reconstruction error (TruncatedSVD):", relative_error)

Relative reconstruction error (TruncatedSVD): 2.6755954945383073e-15


In [ ]:
np.allclose(sparse.toarray(), reconstructed)